In [1]:
def getLabelList(rootPath): 
    list = []
    for filename in os.listdir(rootPath):
        file_path = os.path.join(rootPath,filename)
        if os.path.isdir(file_path):
            list.append(file_path)
            getLabelList(file_path)
    return list

#특정 디렉토리내의 하위 폴더경로를 리스트로 반환해주는 함수

# 사용법
### 1. 코드 23번째 줄 `rootPath 변수에 에  aihub의 영상의 폴더명을 적어줄 것`  ex) aihub
### 2. aihub내에 학습시킬 라벨명에 해당하는 폴더를 만들고 해당 폴더안에 영상을 넣기 
### ex) cat ,dog , ..
### 3. 코드를 돌리면 dataset 폴더에 해당하는 데이터셋이 생김
### +) 속도를 높이고 싶다면 굳이 영상을 확인할필요가 없으므로 76번째 라인을 주석처리 할 것

In [ ]:
import numpy as np
import time, os
import mediapipe as mp
import cv2


seq_length = 30 #하나의 데이터가 가지는 sequence길이 (30개씩 쪼개서 학습시킨다) -> LSTM 사용시 필요함
secs_for_action = 30 #각 데이터당 30초씩 학습을 시킴 //동영상의 길이로 수정할것

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)



created_time = int(time.time())
os.makedirs('dataset', exist_ok=True) #dataSet 저장 폴더 생성

rootPath = "aihub" #영상이 저장된 루트 경로
labelList = getLabelList(rootPath) 

for idx, action in enumerate(labelList): #폴더의 개수만큼 반복해야함. 각 폴더가 라벨명을 의미하도록 수정
        
        for filename in os.listdir(action):
            f = os.path.join(action, filename)
            print(f + "영상에 대한 학습을 진행중입니다.")

            # 내부의 for문을 만들고 특정 폴더내의 모든 영상이 다 사용될때까지 반복
            cap = cv2.VideoCapture(f) #폴더 내 영상을 열기
            data = []
            start_time = time.time()

            while cap.isOpened(): #각 프레임을 읽어 mediaPipe에 넣는다
                ret, img = cap.read()
                if not ret:
                    break;

                img = cv2.flip(img, 1)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                result = hands.process(img)
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

                if result.multi_hand_landmarks is not None:
                    for res in result.multi_hand_landmarks:
                        joint = np.zeros((21, 4))
                        for j, lm in enumerate(res.landmark):
                            joint[j] = [lm.x, lm.y, lm.z, lm.visibility] # 각 각도정보 및 손가락의 랜드마크가 보이는 여부를 추가

                        # Compute angles between joints
                        v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint (각도를 구한다)
                        v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                        v = v2 - v1 # [20, 3]
                        # Normalize v
                        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                        # Get angle using arcos of dot product
                        angle = np.arccos(np.einsum('nt,nt->n',
                            v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                            v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                        angle = np.degrees(angle) # Convert radian to degree

                        angle_label = np.array([angle], dtype=np.float32)
                        angle_label = np.append(angle_label, idx) #idx (=라벨)을 넣어준다.

                        d = np.concatenate([joint.flatten(), angle_label]) #100개의 행렬이 완성

                        data.append(d)

                        mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

                cv2.imshow('img', img) #해당코드 주석처리하면 속도 더 빠름
                if cv2.waitKey(1) == ord('q'):
                    break

        data = np.array(data)
        labelName = os.path.basename(action) 
        print(action, data.shape) #30초 동안 모은 데이터를 np배열로 변환
      
        # Create sequence data (30개씩 모아 시퀸스 데이터를 만든다.)
        full_seq_data = []
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])

        full_seq_data = np.array(full_seq_data)
        print(labelName, "에 대한 seq파일을 정상적으로 생성하였습니다.")
        np.save(os.path.join('dataset', f'seq_{labelName}_{created_time}'), full_seq_data)

        # 즉 raw, seq 2개의 파일을 생성한다. (실제 seq파일을 사용하게됨)
 